
<span type="title">Servlet 和 JSP 概要</span> | <span type="update">2018-09-16</span> - Version <span type="version">1.0</span>
    
    
<span type="intro"><p class="card-text">本章主要对 Servlet 和 JSP 有一个概要性的介绍。</p></span>

# HTTP

互联网通信基于 请求 和 响应 这两大动作。HTTP是一种协议，其在 RFC2616 草稿中规范，大概包括三大重要部分：客户端发送请求时的地址行和发送方法POST,GET等，服务端返回请求时的状态200，400，404等、返回请求时的报文类型以及报文内容。其中报文内容由HTML语言进行标记。

WEB指的就是HTTP和HTML为主体的层的活动。其中互联网请求多对应HTTP的GET和POST方法，响应则多对应于200,300,400等相应类型以及text/html,text/gzip等返回类型和HTML格式或其他格式的文档。

HTTP是一种应用层的协议，其建立在IP/TCP的基础上，IP负责规划和寻找客户端和服务端，TCP负责在线路上分批传送信息以及保证信息的准确性和完整性。HTTP得到的是由TCP生成的完整的报文，其只规定了在多台计算机间传送文档的格式（HTML）以及传送时的一些问题（各种传送状态、传送文档的地址、格式等）。

## HTTP URL

HTTP定义了URI，其中包含URL和URN。URL的格式为：`协议名称:// 子级地址 + [.] 上一级地址 + [.] 主地址 + [.] + 域名后缀:[端口地址]/ 文档路径/ 文档`比如：`http://www.mazhangjing.com/index/hello.txt` 其中协议是 http，域名是 mazhangjing.com（包含主地址和其后缀）。主机名是 www，文档路径为 index 文件夹，文件为此文件夹中的hello.txt文档。其中子级地址可以省略：`http://mazhangjing.com` 文档路径和文档名称都可以省略，并且设置默认值。端口地址默认为80，可以另行指定。此外，URL还包含可选的查询字符串，当使用GET方法时可以少量的添加请求的信息。（见下文）

HTTP的端口是一个16位数，其中1-1023被保留用作它用。一般而言，用于特定用途的服务会默认使用特定的端口，这是一种约定俗成的标准，比如SSH的22端口，HTTP的80和8080端口，HTTPS的443端口和FTP的21端口。

## HTTP GET/POST

对于一个地址，可以使用GET和POST方法，其中GET多用于超链接点击，其可以传送少量参数，并且会显示在地址栏中，容易暴露，比如：`http://www.mazhangjing.com/weather?location=wuhan&time=now`，当对于 `http://www.mazhangjing.com/weather` 发送GET方法时，如果传递了参数，会显示在URL上，这里明显看出参数有两个，location是wuhan，time是now，各个参数用&分隔，参数和文档用？分隔。

POST方法则不会暴露这些信息，并且可以传送大量的数据到服务器，比如文件上传等，因此多用于登录，提交信息等。POST传递参数和GET一样，除了不会直接显示之外，使用F12开发者工具（Chrome/FireFox）可以查看POST发送的数据，注意其没有被显示在地址栏中。


## HTTP RESPONSE

当我们发送GET/POST请求时，会收到来自服务器返回的响应，称之为 HTTP RESPONSE,其中重要的信息是响应代码和类型以及返回的文件格式以及文件内容本身。比如：`HTTP1.1 200 OK`，代码200，正常响应请求。`Content-Type：text/html`，表示返回的是一个HTML文档。


## HTTP 请求的响应流程

当我们指定GET一个网址，客户端会使用HTTP协议发送这个请求，交给TCP到IP，然后经过物理层，找到URL中对应的主机，逆向从IP到TCP到服务端的HTTP，之后WEB服务器监听的端口监听到请求，找到URL路径上的文件，处理可能有的参数，并且返回一个响应，这个响应包含了响应的状态和状态码，以及返回的文档类型以及文档本身。返回的流程和获得的流程相反，从服务端的HTTP到IP到TCP，然后逆向到请求端的TCP到请求端的IP到请求端的HTTP，这样客户端就收到了服务器的文档。

注意，服务端是如何知道客户端的地址的呢？这是通过IP协议规定的，当发送请求时，服务端会得到请求主体的信息，包括其请求的时间以及请求的内容，还有请求主体的IP地址，知道这个IP，就可以返回到natai



# HTML

HTML是一种已经预先定义好的格式标记语言，
